Used following code for ref:

*   https://github.com/amir-cardiolab/PINN_multiphysics_multifidelity/blob/main/2D_Lid_driven_cavity/Lid_driven_cavity_Initialization.py
*   https://github.com/okada39/pinn_cavity/blob/master/main.py
*   NS Youtube: https://github.com/ComputationalDomain/PINNs



# EQUATIONS:
*   **Eqn:**

*   u_x + v_y = 0,
*   u*u_x + v*u_y + p_x/rho - nu*(u_xx + u_yy) = 0,
*   u*v_x + v*v_y + p_y/rho - nu*(v_xx + v_yy) = 0,




*   **BCS:**

*   x, y = 0 ~ 1:
*   u=1, v=0 at top boundary
*   u=0, v=0 at other boundaries











# IMPORTS:

In [ ]:
import torch
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from math import exp, sqrt,pi

import tensorflow as tf

import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.gridspec import GridSpec

import tqdm as tqdm

# CONSTANTS

In [ ]:
rho = 1
nu = 0.01

num_train_samples = 10000
num_bc_samples = 2500
epochs = 1000
lr = 0.001

load_checkpoint = True

# MODEL

In [ ]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 20), nn.Tanh(),
            nn.Linear(20, 2)
        )

    def forward(self, x, y):
        input = torch.cat([x,y], axis = 1)
        output = self.net(input)
        #output = self.net(input)
        return output


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = Net()
net = net.to(device)

mse_loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters())

In [ ]:
def checkpoint(state,filename="checkpt.pth"):
     print('\n',"Saving checkpoint")
     torch.save(state,filename)

def load_checkpt(checkpt):
     print('\n',"Loading checkpoint")
     checkpt = torch.load("checkpt.pth")
     net.load_state_dict(checkpt['state_dict'])
     optimizer.load_state_dict(checkpt['optimizer_dict'])
     net.train()

# PDE

In [ ]:
def Pde(x,y,net):
  #print('psi from pde fun', '\n', psi)

  u = torch.autograd.grad(psi, y, grad_outputs=torch.ones_like(psi), create_graph=True)[0]
  v = -1 * torch.autograd.grad(psi, x, grad_outputs=torch.ones_like(psi), create_graph=True)[0]

  u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
  u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
  u_y = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(u), create_graph=True)[0]
  u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), create_graph=True)[0]


  v_x = torch.autograd.grad(v, x, grad_outputs=torch.ones_like(v), create_graph=True)[0]
  v_xx = torch.autograd.grad(v_x, x, grad_outputs=torch.ones_like(v_x), create_graph=True)[0]
  v_y = torch.autograd.grad(v, y, grad_outputs=torch.ones_like(v), create_graph=True)[0]
  v_yy = torch.autograd.grad(v_y, y, grad_outputs=torch.ones_like(v_y), create_graph=True)[0]


  p_x = torch.autograd.grad(p, x, grad_outputs=torch.ones_like(p), create_graph=True)[0]
  p_y = torch.autograd.grad(p, y, grad_outputs=torch.ones_like(p), create_graph=True)[0]

  continuity_pde_out = u_x + v_y
  x_dir_ns_pde_out = u*u_x + v*u_y + p_x/rho - nu*(u_xx + u_yy)
  y_dir_ns_pde_out = u*v_x + v*v_y + p_y/rho - nu*(v_xx + v_yy)

  pde_out = [continuity_pde_out, x_dir_ns_pde_out, y_dir_ns_pde_out, u, v]
  return pde_out

In [ ]:
def get_uv(x,y,net_out):
  psi, p = torch.split(net_out, split_size_or_sections=1, dim=1)

  u = torch.autograd.grad(psi, y, grad_outputs=torch.ones_like(psi), create_graph=True)[0]
  v = -1 * torch.autograd.grad(psi, x, grad_outputs=torch.ones_like(psi), create_graph=True)[0]

  predicted_vel = torch.cat([u,v], axis = 1)
  return predicted_vel

# BOUNDARY CONDITIONS:

In [ ]:
# Lid BCs:
lid_train_pts = np.random.rand(num_bc_samples, 2)
lid_train_pts[:,1:2] = 1

lid_labels = np.zeros((num_bc_samples,2))
lid_labels[:,0:1] = 1

lid_train_pts_device = torch.from_numpy(lid_train_pts).float().to(device)
lid_train_pts_device.requires_grad = True
x_lid, y_lid = torch.split(lid_train_pts_device, split_size_or_sections=1, dim=1)  # input data for lid bc to network

lid_labels_device = torch.from_numpy(lid_labels).float().to(device)


# Left wall BCs:
lw_train_pts = np.random.rand(num_bc_samples, 2)
lw_train_pts[:,0:1] = 0

lw_labels = np.zeros((num_bc_samples,2))

lw_train_pts_device = torch.from_numpy(lw_train_pts).float().to(device)
lw_train_pts_device.requires_grad = True
x_lw, y_lw = torch.split(lw_train_pts_device, split_size_or_sections=1, dim=1)    # input data for lw bc to  network

lw_labels_device = torch.from_numpy(lw_labels).float().to(device)


# Right wall BCs:
rw_train_pts = np.random.rand(num_bc_samples, 2)
rw_train_pts[:,0:1] = 1

rw_labels = np.zeros((num_bc_samples,2))

rw_train_pts_device = torch.from_numpy(rw_train_pts).float().to(device)
rw_train_pts_device.requires_grad = True
x_rw, y_rw = torch.split(rw_train_pts_device, split_size_or_sections=1, dim=1)    # input data for rw bc to  network

rw_labels_device = torch.from_numpy(rw_labels).float().to(device)


# Lower wall BCs:
btm_train_pts = np.random.rand(num_bc_samples, 2)
btm_train_pts[:,1:2] = 0

btm_labels = np.zeros((num_bc_samples,2))

btm_train_pts_device = torch.from_numpy(btm_train_pts).float().to(device)
btm_train_pts_device.requires_grad = True
x_btm, y_btm = torch.split(btm_train_pts_device, split_size_or_sections=1, dim=1)    # input data for lower wall bc to  network

btm_labels_device = torch.from_numpy(btm_labels).float().to(device)


# CREATING TRAINING POINTS:

In [ ]:
train_pts = np.random.rand(num_train_samples, 2)
train_pts_device = torch.from_numpy(train_pts).float().to(device)
train_pts_device.requires_grad = True

train_labels = np.zeros((num_train_samples,1))
train_labels_device = torch.from_numpy(train_labels).float().to(device)

# Training Loop:

In [ ]:
exp_train = np.random.rand(num_bc_samples, 2)
exp_train_device = torch.from_numpy(exp_train).float().to(device)

x_exp, y_exp = torch.split(exp_train_device, split_size_or_sections=1, dim=1)
exp_out1 = net(x_exp, y_exp)
print(exp_out1)

tensor([[1.0000e+00, 2.3842e-07],
        [1.0000e+00, 2.0862e-07],
        [1.0000e+00, 2.6822e-07],
        ...,
        [1.0000e+00, 2.3842e-07],
        [1.0000e+00, 2.2352e-07],
        [1.0000e+00, 2.9802e-07]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [ ]:
if load_checkpoint == True:
  checkpoint = torch.load('net.pth', map_location = device)
  net.load_state_dict(checkpoint)

In [ ]:
for epoch in range(epochs):

  optimizer.zero_grad()



  #----------------------------BOUNDARY CONDITION LOSS CALCULATION------------------------------#

  # lid:
  net_lid_bc_out = net(x_lid, y_lid)

  lid_predicted_vel = get_uv(x_lid, y_lid, net_lid_bc_out)
  j_lid = mse_loss_fn(lid_predicted_vel, lid_labels_device)

  # left-wall:
  net_lw_bc_out = net(x_lw, y_lw)
  lw_predicted = get_uv(x_lw, y_lw, net_lw_bc_out)
  j_lw = mse_loss_fn(lw_predicted, lw_labels_device)

  # right-wall:
  net_rw_bc_out = net(x_rw, y_rw)
  rw_predicted = get_uv(x_rw, y_rw, net_rw_bc_out)
  j_rw = mse_loss_fn(rw_predicted, rw_labels_device)

  # lower wall:
  net_btm_bc_out = net(x_btm, y_btm)
  btm_predicted = get_uv(x_btm, y_btm, net_btm_bc_out)
  j_btm = mse_loss_fn(btm_predicted, btm_labels_device)


  J_BC = j_lid + j_lw + j_rw + j_btm    # total BC loss
  #----------------------------------------------------------------------------------------------#



  #----------------------------PDE LOSS CALCULATION------------------------------#
  xy = train_pts_device
  x, y = torch.split(xy, split_size_or_sections=1, dim=1)    # input data for lw bc to  network
  xy.requires_grad = True

  output = net(x,y)
  psi = output[:,0:1]
  #print('psi from output slice', '\n', psi)
  p = output[:,1:2]

  pde_outputs = Pde(x,y,net)

  j_continuity = mse_loss_fn(pde_outputs[0], train_labels_device)
  j_ns_x = mse_loss_fn(pde_outputs[1], train_labels_device)
  j_ns_y = mse_loss_fn(pde_outputs[2], train_labels_device)


  J_PDE = j_continuity + j_ns_x + j_ns_y
  #----------------------------------------------------------------------------------------------#



  total_loss = J_BC + J_PDE

  total_loss.backward()

  optimizer.step()

  with torch.autograd.no_grad():
    	print(epoch,"Training Loss:", total_loss.data)

  if epoch % 100 == 0:
    torch.save(net.state_dict(), 'net.pth')





0 Training Loss: tensor(0.0386)
1 Training Loss: tensor(0.2652)
2 Training Loss: tensor(0.0665)
3 Training Loss: tensor(0.1610)
4 Training Loss: tensor(0.0998)
5 Training Loss: tensor(0.0402)
6 Training Loss: tensor(0.0630)
7 Training Loss: tensor(0.0915)
8 Training Loss: tensor(0.0948)
9 Training Loss: tensor(0.0770)
10 Training Loss: tensor(0.0537)
11 Training Loss: tensor(0.0435)
12 Training Loss: tensor(0.0545)
13 Training Loss: tensor(0.0699)
14 Training Loss: tensor(0.0676)
15 Training Loss: tensor(0.0533)
16 Training Loss: tensor(0.0450)
17 Training Loss: tensor(0.0478)
18 Training Loss: tensor(0.0550)
19 Training Loss: tensor(0.0592)
20 Training Loss: tensor(0.0573)
21 Training Loss: tensor(0.0512)
22 Training Loss: tensor(0.0456)
23 Training Loss: tensor(0.0450)
24 Training Loss: tensor(0.0490)
25 Training Loss: tensor(0.0520)
26 Training Loss: tensor(0.0498)
27 Training Loss: tensor(0.0453)
28 Training Loss: tensor(0.0432)
29 Training Loss: tensor(0.0446)
30 Training Loss: te

KeyboardInterrupt: ignored

# Plotting:

In [ ]:
x_coords = np.linspace(0, 1, num_train_samples)
y_coords = np.linspace(0, 1, num_train_samples)

x_m, y_m = np.meshgrid(x_coords, y_coords)
xy_coords = np.stack([x.flatten(), y.flatten()], axis=-1)

(10000, 10000)
